# ОДИНОЧКА

## Суть паттерна

<b>Одиночка</b> - это <i>порождающий</i> паттерн проектирования, который гарантирует, что у класса есть только один экземпляр, и предоставляет к нему глобальную точку доступа.

<img src="./images/singleton.png"/>

## Проблема

<b>Одиночка</b> решает сразу две проблемы, нарушая <i>принцип единственной ответственности</i> класса.
1. <b>Гарантирует наличие единственного экземпляра класса.</b> Чаще всего это полезно для доступа к какому-то общему ресурсу, например, базе данных. Представьте, что вы создали объект, а через некоторое время пробуете создать ещё один. В этом случае хотелось бы получить старый объект, вместо создания нового.

2. <b>Предоставляет глобальную точку доступа.</b> Это не просто глобальная переменная, через которую можно достучаться к определённому объекту. Глобальные переменные не защищены от записи, поэтому любой код может подменять их значения без вашего ведома. Но есть и другой нюанс. Неплохо бы хранить в одном месте и код, который решает проблему №1, а также иметь к нему простой и доступный интерфейс.

## Решение

Все реализации одиночки сводятся к тому, чтобы скрыть конструктор по умолчанию и создать публичный статический метод, который и будет контролировать жизненный цикл объекта-одиночки. Если у вас есть доступ к классу одиночки, значит, будет доступ и к этому статическому методу. Из какой точки кода вы бы его ни вызвали, он всегда будет отдавать один и тот же объект.

## Аналогия из жизни

Правительство государства — хороший пример одиночки. В государстве может быть только одно официальное правительство. Вне зависимости от того, кто конкретно заседает в правительстве, оно имеет глобальную точку доступа «Правительство страны N».

## Структура

<img src="./images/singleton_struct.png"/>

1. <b>Одиночка</b> определяет статический метод getInstance, который возвращает единственный экземпляр своего класса. Конструктор одиночки должен быть скрыт от клиентов. Вызов метода getInstance должен стать единственным способом получить объект этого класса.

## Код

### Наивный одиночка

In [13]:
from typing import Optional


class SingletonMeta(type):
    """
    В Python класс Одиночка можно реализовать по-разному. Возможные способы
    включают себя базовый класс, декоратор, метакласс. Мы воспользуемся
    метаклассом, поскольку он лучше всего подходит для этой цели.
    """

    _instance: Optional[Singleton] = None

    def __call__(self) -> Singleton:
        if self._instance is None:
            self._instance = super().__call__()
        return self._instance


class Singleton(metaclass=SingletonMeta):
    def some_business_logic(self):
        """
        Наконец, любой одиночка должен содержать некоторую бизнес-логику,
        которая может быть выполнена на его экземпляре.
        """


if __name__ == "__main__":
    # Клиентский код.

    s1 = Singleton()
    s2 = Singleton()

    if id(s1) == id(s2):
        print("Singleton works, both variables contain the same instance.")
    else:
        print("Singleton failed, variables contain different instances.")

Singleton works, both variables contain the same instance.


### Многопоточный одиночка

In [14]:
from threading import Lock, Thread
from typing import Optional


class SingletonMeta(type):
    """
    Это потокобезопасная реализация класса Singleton.
    """

    _instance: Optional[Singleton] = None

    _lock: Lock = Lock()
    """
    У нас теперь есть объект-блокировка для синхронизации потоков во время
    первого доступа к Одиночке.
    """

    def __call__(cls, *args, **kwargs):
        # Теперь представьте, что программа была только-только запущена.
        # Объекта-одиночки ещё никто не создавал, поэтому несколько потоков
        # вполне могли одновременно пройти через предыдущее условие и достигнуть
        # блокировки. Самый быстрый поток поставит блокировку и двинется внутрь
        # секции, пока другие будут здесь его ожидать.
        with cls._lock:
            # Первый поток достигает этого условия и проходит внутрь, создавая
            # объект-одиночку. Как только этот поток покинет секцию и освободит
            # блокировку, следующий поток может снова установить блокировку и
            # зайти внутрь. Однако теперь экземпляр одиночки уже будет создан и
            # поток не сможет пройти через это условие, а значит новый объект не
            # будет создан.
            if not cls._instance:
                cls._instance = super().__call__(*args, **kwargs)
        return cls._instance


class Singleton(metaclass=SingletonMeta):
    value: str = None
    """
    Мы используем это поле, чтобы доказать, что наш Одиночка действительно
    работает.
    """

    def __init__(self, value: str) -> None:
        self.value = value

    def some_business_logic(self):
        """
        Наконец, любой одиночка должен содержать некоторую бизнес-логику,
        которая может быть выполнена на его экземпляре.
        """


def test_singleton(value: str) -> None:
    singleton = Singleton(value)
    print(singleton.value)


if __name__ == "__main__":
    # Клиентский код.

    print("If you see the same value, then singleton was reused (yay!)\n"
          "If you see different values, then 2 singletons were created (booo!!)\n\n"
          "RESULT:\n")

    process1 = Thread(target=test_singleton, args=("FOO",))
    process2 = Thread(target=test_singleton, args=("BAR",))
    process1.start()
    process2.start()

If you see the same value, then singleton was reused (yay!)
If you see different values, then 2 singletons were created (booo!!)

RESULT:

FOO
FOO


## Применимость

<b>Когда в программе должен быть единственный экземпляр какого-то класса, доступный всем клиентам (например, общий доступ к базе данных из разных частей программы).</b>

Одиночка скрывает от клиентов все способы создания нового объекта, кроме специального метода. Этот метод либо создаёт объект, либо отдаёт существующий объект, если он уже был создан.

<b>Когда вам хочется иметь больше контроля над глобальными переменными.</b>

В отличие от глобальных переменных, Одиночка гарантирует, что никакой другой код не заменит созданный экземпляр класса, поэтому вы всегда уверены в наличии лишь одного объекта-одиночки.

Тем не менее, в любой момент вы можете расширить это ограничение и позволить любое количество объектов-одиночек, поменяв код в одном месте (метод getInstance).

## Шаги реализации

1. Добавьте в класс приватное статическое поле, которое будет содержать одиночный объект.

2. Объявите статический создающий метод, который будет использоваться для получения одиночки.

3. Добавьте «ленивую инициализацию» (создание объекта при первом вызове метода) в создающий метод одиночки.

4. Сделайте конструктор класса приватным.

5. В клиентском коде замените вызовы конструктора одиночка вызовами его создающего метода.

## Преимущества и недостатки

\+ Гарантирует наличие единственного экземпляра класса.

\+ Предоставляет к нему глобальную точку доступа.

\+ Реализует отложенную инициализацию объекта-одиночки.

\- Нарушает принцип единственной ответственности класса.

\- Маскирует плохой дизайн.

\- Проблемы мультипоточности.

\- Требует постоянного создания Mock-объектов при юнит-тестировании.

## Отношения с другими паттернами

- <b>Фасад</b> можно сделать <b>Одиночкой</b>, так как обычно нужен только один объект-фасад.

- Паттерн <b>Легковес</b> может напоминать <b>Одиночку</b>, если для конкретной задачи у вас получилось свести количество объектов к одному. Но помните, что между паттернами есть два кардинальных отличия:

> 1. В отличие от Одиночки, вы можете иметь множество объектов-легковесов.
> 2. Объекты-легковесы должны быть неизменяемыми, тогда как объект-одиночка допускает изменение своего состояния.

- <b>Абстрактная фабрика, Строитель</b> и <b>Прототип</b> могут быть реализованы при помощи <b>Одиночки</b>.